In [45]:
# pip install pandas

In [46]:
# pip install numpy

In [47]:
# pip install folium

In [48]:
import pandas as pd
import numpy as np
import random

# Create a mock dataset
np.random.seed(16)

n_schools = 40
schools = ['School_' + str(i) for i in range(n_schools)]
latitudes = np.random.uniform(low=42.5, high=45.2, size=n_schools)
longitudes = np.random.uniform(low=15.7, high=19.6, size=n_schools)
internet_status = np.random.choice(['Yes', 'No'], n_schools)
roma_students = np.random.randint(0, 100, n_schools)
total_students = np.random.randint(100, 1000, n_schools)
rural_zone = np.random.choice(['Yes', 'No'], n_schools)

# Create DataFrame
data = {
    'School Name': schools,
    'Latitude': latitudes,
    'Longitude': longitudes,
    'Internet Connectivity': internet_status,
    'Roma Students': roma_students,
    'Total Students': total_students,
    'Rural Zone': rural_zone
}

df = pd.DataFrame(data)
df['Roma Percentage'] = (df['Roma Students'] / df['Total Students']) * 100
df.head()


,School Name,Latitude,Longitude,Internet Connectivity,Roma Students,Total Students,Rural Zone,Roma Percentage
0,School_0,43.102886,18.708048,Yes,94,603,Yes,15.588723
1,School_1,43.912541,18.882955,No,38,217,Yes,17.511521
2,School_2,43.986894,16.244602,Yes,18,587,No,3.066440
3,School_3,42.623125,17.954226,No,86,303,No,28.382838
4,School_4,43.473968,17.028630,No,49,185,No,26.486486


In [49]:
# Summary statistics
df.describe()

,Latitude,Longitude,Roma Students,Total Students,Roma Percentage
count,40.000000,40.000000,40.000000,40.000000,40.000000
mean,43.645831,17.639388,53.100000,466.875000,15.536752
std,0.729836,1.079943,28.393571,248.438424,13.200028
min,42.530993,15.771760,2.000000,116.000000,0.776197
25%,43.046020,16.603876,30.000000,282.750000,5.305314
50%,43.729972,17.833286,54.500000,435.500000,9.937226
75%,44.301037,18.529077,74.250000,643.500000,21.408769
max,45.040729,19.509726,99.000000,982.000000,56.395349


In [50]:
# Identify exclusion based on Roma percentage and rural status
df['High Exclusion Risk'] = np.where((df['Roma Percentage'] > 5) & (df['Rural Zone'] == 'Yes') & (df['Internet Connectivity'] == 'No'), 'Yes', 'No')

df[['School Name', 'High Exclusion Risk', 'Roma Percentage', 'Rural Zone', 'Internet Connectivity']].head()

,School Name,High Exclusion Risk,Roma Percentage,Rural Zone,Internet Connectivity
0,School_0,No,15.588723,Yes,Yes
1,School_1,Yes,17.511521,Yes,No
2,School_2,No,3.066440,No,Yes
3,School_3,No,28.382838,No,No
4,School_4,No,26.486486,No,No


In [51]:
import folium
from folium.plugins import MarkerCluster

# Create base map
map_exclusion = folium.Map(location=[44, 17.5], zoom_start=7)

# Add markers
marker_cluster = MarkerCluster().add_to(map_exclusion)

from folium import IFrame

# Iterate over your dataset and create improved popups
for i, row in df.iterrows():
    # Create the HTML content for the popup
    popup_content = f"""
    <div style="font-family: Arial, sans-serif; font-size: 14px; line-height: 1.5;">
        <strong>School:</strong> {row['School Name']}<br>
        <strong>Roma Students:</strong> {row['Roma Students']}<br>
        <strong>Total Students:</strong> {row['Total Students']}<br>
        <strong>Rural Zone:</strong> {row['Rural Zone']}<br>
        <strong>Internet Connectivity:</strong> {row['Internet Connectivity']}
    </div>
    """
    
    # Create an IFrame to hold the HTML content
    iframe = IFrame(html=popup_content, width=250, height=150)
    
    # Create the popup and add it to the map
    popup = folium.Popup(iframe, max_width=250)
    color = 'red' if row['High Exclusion Risk'] == 'Yes' else 'green'
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)


# Display the map
map_exclusion

In [52]:
map_exclusion.save("map.html")

# Short Conclusion Example

After performing the analysis, we identified that **X** schools are at high risk of exclusion. The key factors for exclusion were:

- A high percentage of Roma students (more than 20%).
- Located in rural areas.
- Lack of internet connectivity.

The map above highlights the schools in red that are considered at high exclusion risk. Further steps should involve prioritizing internet access for these schools and targeting support programs for Roma students to reduce inequalities.
